In [1]:
from src.preprocessing import read, tokenize
from src.helpers import tokens_to_term_freqs
from src.database import Database

In [2]:
test_db_tokens = {
    "doc1": ["hey", "hello", "how", "are", "you", "hello", "hello"],
    "doc2": ["what", "is", "your", "name"],
    "doc3": ["hey", "how", "are", "you", "doing"]
}

test_db = {
    "doc1": "hey hello how are you hello hello",
    "doc2": "what is your name",
    "doc3": "hey how are you doing"
}


def tokenize(text):
    return text.split()

for t in test_db.values():
    print(tokenize(t))

['hey', 'hello', 'how', 'are', 'you', 'hello', 'hello']
['what', 'is', 'your', 'name']
['hey', 'how', 'are', 'you', 'doing']


In [3]:
db = Database(tokenize_fn=tokenize, docs=test_db)


db.query("hello how are you")

[('doc1', 0.9871715551505026), ('doc3', 0.5386043776164267)]